In [38]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [39]:
# Load data and sort them by time
def load_data(data_path):
    df = pd.read_csv(data_path)
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df.sort_values(by=['DateTime'], inplace=True)
    return df

In [40]:
def preprocess_data(df):
    # Encode Junction as a numerical feature
    junction_encoder = LabelEncoder()
    df['Junction_encoded'] = junction_encoder.fit_transform(df['Junction'])
    
    # Extract time-based features for better pattern recognition
    df['Hour'] = df['DateTime'].dt.hour
    df['DayOfWeek'] = df['DateTime'].dt.dayofweek
    df['Month'] = df['DateTime'].dt.month
    df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
    
    # Normalize data
    scaler_dict = {}
    numerical_features = ['Vehicles', 'Hour', 'DayOfWeek', 'Month', 'Junction_encoded']
    for feature in numerical_features:
        scaler = MinMaxScaler()
        df[f'{feature}_normalized'] = scaler.fit_transform(df[[feature]])
        scaler_dict[feature] = scaler
    
    return df, scaler_dict, junction_encoder

In [41]:
# LSTM model with multi-step prediction
class JunctionTrafficLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=32, num_layers=5, output_size=1):
        super(JunctionTrafficLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
    
    def forward(self, x, future_steps=1, teacher_forcing_ratio=0.5):
        batch_size = x.size(0)
        outputs = []
        lstm_out, (h, c) = self.lstm(x)
        decoder_input = x[:, -1:, :]
        
        for i in range(future_steps):
            # Run LSTM step
            out, (h, c) = self.lstm(decoder_input, (h, c))
            
            # Process through
            features = self.dropout(out[:, -1])
            features = self.relu(self.fc1(features))
            features = self.relu(self.fc2(features))
            prediction = self.fc3(features)
            
            outputs.append(prediction)
        
            if self.training and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input = x[:, i+1:i+2, :] if i+1 < x.size(1) else decoder_input
            else:
                next_input = decoder_input.clone()
                next_input[:, 0, 0] = prediction.squeeze()
                decoder_input = next_input
        
        return torch.stack(outputs, dim=1)

In [42]:
def create_multistep_sequences(data, input_seq_length, prediction_length):
    feature_cols = [col for col in data.columns if '_normalized' in col]
    sequences = []
    targets = []
    
    # Group by junction to ensure sequences don't cross junction boundaries
    for junction in data['Junction'].unique():
        junction_data = data[data['Junction'] == junction].copy()
        
        for i in range(len(junction_data) - input_seq_length - prediction_length):
            seq = junction_data[feature_cols].iloc[i:i+input_seq_length].values
            target = junction_data['Vehicles_normalized'].iloc[i+input_seq_length:i+input_seq_length+prediction_length].values
            
            sequences.append(seq)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

In [43]:
def train_multistep_model(model, train_loader, val_loader, criterion, optimizer, model_path='best_model.pth', 
                          num_epochs=50, patience=10, prediction_length=24, device='cpu'):
    best_val_loss = float('inf')
    patience_counter = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            
            # Generate predictions
            outputs = model(batch_X, future_steps=prediction_length)
            
            # Ensure outputs and batch_y have the same shape
            outputs = outputs.squeeze()
            if len(outputs.shape) == 1:
                outputs = outputs.unsqueeze(0)
            if len(batch_y.shape) == 1:
                batch_y = batch_y.unsqueeze(0)

            loss = criterion(outputs, batch_y)

            # Add consistency loss to ensure smooth predictions
            if outputs.size(1) > 1:
                output_diff = outputs[:, 1:] - outputs[:, :-1]
                target_diff = batch_y[:, 1:] - batch_y[:, :-1]
                
                min_length = min(output_diff.size(1), target_diff.size(1))
                consistency_loss = criterion(
                    output_diff[:, :min_length],
                    target_diff[:, :min_length]
                )
                loss = loss + 0.1 * consistency_loss
            
            loss.backward()
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X, future_steps=prediction_length)
                
                outputs = outputs.squeeze()
                if len(outputs.shape) == 1:
                    outputs = outputs.unsqueeze(0)
                if len(batch_y.shape) == 1:
                    batch_y = batch_y.unsqueeze(0)
                    
                val_loss += criterion(outputs, batch_y).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break
    
    # Plot training curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training Progress')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('training_progress.png')
    plt.close()
    
    # Load best model
    model.load_state_dict(torch.load(model_path))
    return model

In [44]:
def accuracy(model, test_loader, threshold, prediction_length=24, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X, future_steps=prediction_length)
            difference = torch.abs(outputs.squeeze() - batch_y)
            correct += (difference <= threshold).sum().item()
            total += batch_y.numel()
    
    accuracy_percentage = (correct / total) * 100
    print(f"Test Accuracy: {accuracy_percentage:.2f}%")

In [45]:
def predict_future_traffic(model, df, latest_time, num_predictions, interval_days, 
                           scalers, junction, seq_length=10, device='cpu'):
    """
    Make predictions for a specific junction with custom intervals
    
    Parameters:
    - model: trained LSTM model
    - df: dataframe with the traffic data
    - latest_time: datetime object for the starting point
    - num_predictions: number of predictions to make
    - interval_days: interval between predictions in days
    - scalers: dictionary of fitted scalers
    - junction: junction identifier
    - seq_length: length of input sequence
    - device: device to run predictions on
    
    Returns:
    - predictions: numpy array of predictions
    - prediction_times: list of datetime objects for the predictions
    """
    model.eval()
    
    # Filter data for specific junction
    junction_data = df[df['Junction'] == junction].copy()
    
    if len(junction_data) < seq_length:
        raise ValueError(f"Not enough data points for junction {junction}. Need at least {seq_length} points.")
    
    # Reset index for the filtered data to avoid indexing issues
    junction_data = junction_data.reset_index(drop=True)
    
    # Find the closest time in our data to the requested time
    closest_time_idx = (junction_data['DateTime'] - latest_time).abs().argmin()
    
    # Check if we have enough data points after the closest time
    if closest_time_idx + seq_length > len(junction_data):
        # If not enough points after, we need to use earlier data
        closest_time_idx = len(junction_data) - seq_length
    
    actual_start_time = junction_data.iloc[closest_time_idx]['DateTime']
    time_diff = actual_start_time - latest_time
    print(f"Time difference from requested: {time_diff}")
    
    # Generate the prediction times
    prediction_times = [actual_start_time + timedelta(days=i*interval_days) for i in range(num_predictions)]
    
    # Get feature columns and input sequence
    feature_cols = [col for col in df.columns if '_normalized' in col]
    input_seq = junction_data.iloc[closest_time_idx:closest_time_idx+seq_length][feature_cols].values
    
    # Make predicted features for each time point
    all_predictions = []
    current_input = input_seq.copy()
    
    # Convert to tensor for the model
    input_tensor = torch.tensor(current_input, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Generate one prediction at a time for more accurate long-term forecasting
    for i in range(num_predictions):
        with torch.no_grad():
            # Predict next value
            prediction = model(input_tensor, future_steps=1)
            prediction_value = prediction.squeeze().cpu().numpy()
            
            # Store the prediction
            all_predictions.append(prediction_value)
            
            # Update input sequence for next prediction if we need more predictions
            if i < num_predictions - 1:
                # Create a copy of the input with the next predicted value
                next_input = current_input.copy()
                # Shift window and add new prediction
                next_input = np.roll(next_input, -1, axis=0)
                next_input[-1, 0] = prediction_value  # Set vehicles normalized value
                
                # For time features, we need to calculate them for the next prediction time
                next_time = prediction_times[i+1]
                next_input[-1, 1] = scalers['Hour'].transform([[next_time.hour]])[0][0]  # Hour
                next_input[-1, 2] = scalers['DayOfWeek'].transform([[next_time.dayofweek]])[0][0]  # DayOfWeek
                next_input[-1, 3] = scalers['Month'].transform([[next_time.month]])[0][0]  # Month
                next_input[-1, 4] = scalers['Junction_encoded'].transform([[junction_data['Junction_encoded'].iloc[0]]])[0][0]  # Junction_encoded
                
                # Update current input for next iteration
                current_input = next_input
                input_tensor = torch.tensor(current_input, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Inverse transform to get actual vehicle numbers
    predictions = scalers['Vehicles'].inverse_transform(np.array(all_predictions).reshape(-1, 1)).flatten()
    
    return predictions, prediction_times

In [46]:
# Visualization function for traffic predictions
def visualize_predictions(predictions, prediction_times, junction, output_dir):
    """
    Create visualizations of traffic predictions
    
    Parameters:
    - predictions: array of predicted vehicle counts
    - prediction_times: list of datetime objects for the predictions
    - junction: junction identifier
    - output_dir: directory to save the visualization
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert prediction times to more readable format
    formatted_times = [time.strftime('%Y-%m-%d') for time in prediction_times]
    
    # Create a DataFrame for easy plotting
    pred_df = pd.DataFrame({
        'DateTime': prediction_times,
        'Predicted_Vehicles': predictions,
        'Junction': junction
    })
    
    # Line plot
    plt.figure(figsize=(12, 6))
    plt.plot(pred_df['DateTime'], pred_df['Predicted_Vehicles'], marker='o', linestyle='-')
    plt.title(f'Predicted Traffic Flow for Junction {junction}')
    plt.xlabel('Date')
    plt.ylabel('Vehicle Count')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/traffic_prediction_line_j{junction}.png')
    plt.close()
    
    # Scatter plot with trend line
    plt.figure(figsize=(12, 6))
    scatter = plt.scatter(
        pred_df['DateTime'], 
        pred_df['Predicted_Vehicles'],
        c=range(len(pred_df)),  # Color by time progression
        cmap='viridis',
        s=100,  # Point size
        alpha=0.7
    )
    
    # Add trend line
    z = np.polyfit(range(len(pred_df)), pred_df['Predicted_Vehicles'], 1)
    p = np.poly1d(z)
    plt.plot(pred_df['DateTime'], p(range(len(pred_df))), "r--", linewidth=2)
    
    # Add colorbar to show time progression
    cbar = plt.colorbar(scatter)
    cbar.set_label('Time Progression')
    
    plt.title(f'Traffic Flow Trend for Junction {junction}')
    plt.xlabel('Date')
    plt.ylabel('Vehicle Count')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/traffic_prediction_scatter_j{junction}.png')
    plt.close()
    
    # Save predictions to CSV
    pred_df.to_csv(f'{output_dir}/predicted_traffic_j{junction}.csv', index=False)
    
    print(f"Visualizations and predictions saved to {output_dir}")
    
    return pred_df

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_path = './Data/traffic.csv'  # Path to your data file
output_dir = './results'  # Output directory for saving results and visualizations

model_path = 'best_model.pth'  # Path to save the model

junction = None  # Define which junction to predict, or set to None to predict for all
prediction_horizon = 365  # Number of days to predict
interval = 7  # Interval between predictions in days
train = True  # Whether to train the model or not

In [49]:
os.makedirs(output_dir, exist_ok=True)
df = load_data(data_path)
df, scaler_dict, junction_encoder = preprocess_data(df)
available_junctions = df['Junction'].unique()
print(f"Available junctions in the dataset: {available_junctions}")

Available junctions in the dataset: [1 2 3 4]


In [50]:
# Training and Prediction Pipeline
input_seq_length = 10  # Length of input sequences
prediction_length = 24  # Number of future steps to predict during training

# Split data into train, validation, and test sets
# We'll use time-based splitting to simulate real-world forecasting
time_sorted_data = df.sort_values('DateTime')
train_size = int(len(time_sorted_data) * 0.7)
val_size = int(len(time_sorted_data) * 0.15)

train_data = time_sorted_data.iloc[:train_size]
val_data = time_sorted_data.iloc[train_size:train_size+val_size]
test_data = time_sorted_data.iloc[train_size+val_size:]

print(f"Train data: {len(train_data)} samples")
print(f"Validation data: {len(val_data)} samples")
print(f"Test data: {len(test_data)} samples")

# Create sequences for training
X_train, y_train = create_multistep_sequences(train_data, input_seq_length, prediction_length)
X_val, y_val = create_multistep_sequences(val_data, input_seq_length, prediction_length)
X_test, y_test = create_multistep_sequences(test_data, input_seq_length, prediction_length)

print(f"Training sequences: {X_train.shape}, targets: {y_train.shape}")
print(f"Validation sequences: {X_val.shape}, targets: {y_val.shape}")
print(f"Test sequences: {X_test.shape}, targets: {y_test.shape}")

# Convert to PyTorch tensors
train_tensor_X = torch.FloatTensor(X_train)
train_tensor_y = torch.FloatTensor(y_train)
val_tensor_X = torch.FloatTensor(X_val)
val_tensor_y = torch.FloatTensor(y_val)
test_tensor_X = torch.FloatTensor(X_test)
test_tensor_y = torch.FloatTensor(y_test)

# Create data loaders
batch_size = 64
train_dataset = TensorDataset(train_tensor_X, train_tensor_y)
val_dataset = TensorDataset(val_tensor_X, val_tensor_y)
test_dataset = TensorDataset(test_tensor_X, test_tensor_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[2]  # Number of features
model = JunctionTrafficLSTM(input_size=input_size, hidden_size=64, num_layers=3, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

Train data: 33684 samples
Validation data: 7218 samples
Test data: 7218 samples
Training sequences: (33548, 10, 5), targets: (33548, 24)
Validation sequences: (7082, 10, 5), targets: (7082, 24)
Test sequences: (7082, 10, 5), targets: (7082, 24)


In [51]:
Train = True
# Training process
if train:
    print("Starting model training...")
    model = train_multistep_model(
        model, 
        train_loader, 
        val_loader, 
        criterion, 
        optimizer,
        model_path=model_path,
        num_epochs=50, 
        patience=8, # modify patience
        prediction_length=prediction_length,
        device=device
    )
    print("Training complete!")
else:
    # Load pre-trained model
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    print(f"Loaded pre-trained model from {model_path}")

# Evaluate the model
print("\nEvaluating model performance...")
threshold = 0.05  # Acceptable error threshold for normalized values
accuracy(model, test_loader, threshold, prediction_length=prediction_length, device=device)

Starting model training...
Epoch 1, Train Loss: 0.0041, Val Loss: 0.0074
Model saved to best_model.pth
Epoch 2, Train Loss: 0.0028, Val Loss: 0.0066
Model saved to best_model.pth
Epoch 3, Train Loss: 0.0019, Val Loss: 0.0042
Model saved to best_model.pth
Epoch 4, Train Loss: 0.0015, Val Loss: 0.0032
Model saved to best_model.pth
Epoch 5, Train Loss: 0.0013, Val Loss: 0.0027
Model saved to best_model.pth
Epoch 6, Train Loss: 0.0013, Val Loss: 0.0029
Epoch 7, Train Loss: 0.0012, Val Loss: 0.0026
Model saved to best_model.pth
Epoch 8, Train Loss: 0.0012, Val Loss: 0.0020
Model saved to best_model.pth
Epoch 9, Train Loss: 0.0012, Val Loss: 0.0022
Epoch 10, Train Loss: 0.0012, Val Loss: 0.0024
Epoch 11, Train Loss: 0.0011, Val Loss: 0.0026
Epoch 12, Train Loss: 0.0011, Val Loss: 0.0021
Epoch 13, Train Loss: 0.0011, Val Loss: 0.0021
Epoch 14, Train Loss: 0.0011, Val Loss: 0.0019
Model saved to best_model.pth
Epoch 15, Train Loss: 0.0011, Val Loss: 0.0018
Model saved to best_model.pth
Epoch 1

In [52]:
# Make predictions for the future
print("\nGenerating future traffic predictions...")
latest_time = df['DateTime'].max()
print(f"Latest data point: {latest_time}")

# Define junctions to predict
if junction is None:
    junctions_to_predict = available_junctions
else:
    junctions_to_predict = [junction]

# Make predictions for each junction
all_predictions = []
for junction in junctions_to_predict:
    print(f"\nPredicting traffic for Junction {junction}...")
    
    predictions, prediction_times = predict_future_traffic(
        model,
        df,
        latest_time,
        num_predictions=prediction_horizon // interval + 1,  # +1 to include the end point
        interval_days=interval,
        scalers=scaler_dict,
        junction=junction,
        seq_length=input_seq_length,
        device=device
    )
    
    # Visualize and save predictions
    pred_df = visualize_predictions(predictions, prediction_times, junction, output_dir)
    all_predictions.append(pred_df)
    
    # Print some prediction statistics
    print(f"Junction {junction} prediction summary:")
    print(f"  Average predicted vehicles: {pred_df['Predicted_Vehicles'].mean():.2f}")
    print(f"  Maximum predicted vehicles: {pred_df['Predicted_Vehicles'].max():.2f}")
    print(f"  Minimum predicted vehicles: {pred_df['Predicted_Vehicles'].min():.2f}")

# Combine all predictions for comparison visualization
if len(all_predictions) > 1:
    all_pred_df = pd.concat(all_predictions)
    
    # Create comparative visualization across junctions
    plt.figure(figsize=(14, 8))
    for junction in junctions_to_predict:
        junction_data = all_pred_df[all_pred_df['Junction'] == junction]
        plt.plot(junction_data['DateTime'], junction_data['Predicted_Vehicles'], 
                 marker='o', linestyle='-', label=f'Junction {junction}')
    
    plt.title('Comparative Traffic Flow Predictions Across Junctions')
    plt.xlabel('Date')
    plt.ylabel('Vehicle Count')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/comparative_traffic_predictions.png')
    plt.close()
    
    print(f"\nComparative visualization saved to {output_dir}/comparative_traffic_predictions.png")

print("\nPrediction process complete!")


Generating future traffic predictions...
Latest data point: 2017-06-30 23:00:00

Predicting traffic for Junction 1...
Time difference from requested: -1 days +15:00:00


c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature nam

Visualizations and predictions saved to ./results
Junction 1 prediction summary:
  Average predicted vehicles: 33.07
  Maximum predicted vehicles: 72.34
  Minimum predicted vehicles: 20.25

Predicting traffic for Junction 2...
Time difference from requested: -1 days +15:00:00


c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature nam

Visualizations and predictions saved to ./results
Junction 2 prediction summary:
  Average predicted vehicles: 10.76
  Maximum predicted vehicles: 24.06
  Minimum predicted vehicles: 9.01

Predicting traffic for Junction 3...
Time difference from requested: -1 days +15:00:00


c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature nam

Visualizations and predictions saved to ./results
Junction 3 prediction summary:
  Average predicted vehicles: 9.63
  Maximum predicted vehicles: 23.10
  Minimum predicted vehicles: 8.15

Predicting traffic for Junction 4...
Time difference from requested: -1 days +15:00:00


c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\tianx\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature nam

Visualizations and predictions saved to ./results
Junction 4 prediction summary:
  Average predicted vehicles: 7.56
  Maximum predicted vehicles: 11.10
  Minimum predicted vehicles: 6.39

Comparative visualization saved to ./results/comparative_traffic_predictions.png

Prediction process complete!
